**Inhalt:** Selbständige Übung in Gruppen

**Nötige Skills:** Time Series

**Lernziele:**
- Selbständig Daten explorieren und Storyideen testen

# Das Beispiel

Covid-Daten aus den Schweizer Kantonen.

- Auswertung einzeln bzw. anhand von Gruppen
- Generierung von Story-Ideen und Plotten

Die Daten stammen von dieser Quelle: https://github.com/openZH/covid_19

# Vorbereitung

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
%matplotlib inline

## Daten laden

### 1. Fallzahlen

Wir können die Daten direkt von Github beziehen

In [ ]:
path = 'https://github.com/openZH/covid_19/raw/master/COVID19_Fallzahlen_CH_total_v2.csv'

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head(5)

In [ ]:
df.tail()

### 2. Hilfsdaten zu Kantonen

Vorbereitetes File mit einigen Angaben, die wir nutzen können

In [ ]:
df_c = pd.read_excel('dataprojects/Covid/cantons.xlsx')

In [ ]:
df_c['bevölkerung'] = df_c['bevölkerung'].astype(int)

In [ ]:
df_c.head()

# Daten checken

In [ ]:
df.dtypes

In [ ]:
df.shape

# Daten arrangieren

Unser Ziel sind zwei Dataframes:
- eines mit den kumulierten Fallzahlen (das sind die Werte, die wir in der Spalte "ncumul_conf" erhalten): `df_kum`
- eines mit der täglichen Indizenz (das müssen wir erst noch berechnen): `df_inz`

Wir wollen beide Dataframes im "wide"-Format haben. Das heisst:
- Die Indexspalte enthält das Datum (für jeden Tag genau eine Zeile)
- Die Kantone sind auf die restlichen Spalten aufgeteilt (AG, AI, AR, ...)

Um an diesen Punkt zu kommen, brauchen wir die Funktion `pivot()`. Sie erfordert drei Angaben:
- `index`: Aus welcher Spalte des originalen DF sollen die **Indexwerte** des neuen DF gebildet werden?
- `columns`: Aus welcher Spalte des orignalen DF sollen die **Spalten** des neuen DF gebildet werden?
- `values`: Aus welcher Spalte des orignalen DF sollen die **Werte** des neuen DF gebildet werden?

In [ ]:
df_kum = df.pivot(index='date', columns='abbreviation_canton_and_fl', values='ncumul_conf')

In [ ]:
df_kum.head()

In [ ]:
df_kum.tail()

### Nullen und NaNs

Ein Problem sind die NaNs - nicht für jeden Tag existiert eine Meldung aus jedem Kanton. Wir müssen die Leerstellen mit dem jeweils vorherigen Wert füllen. Dazu benutzen wir `.ffill()`

In [ ]:
df_kum = df_kum.ffill()

In [ ]:
df_kum.head()

In [ ]:
df_kum.tail()

Nächster Schritt: Die NaN am Anfang müssen durch Nullen ersetzt werden.

In [ ]:
df_kum = df_kum.fillna(0).astype(int)

In [ ]:
df_kum.head()

### Indexspalte

Datentyp auf 'datetime':

In [ ]:
df_kum.index = pd.to_datetime(df_kum.index)

... und noch etwas Verschönerung:

In [ ]:
df_kum.index.name = None
df_kum.columns.name = None

### Inzidenzen

Um aus den kumulierten Fallzahlen die täglichen Inzidenzen zu erhalten, nutzen wir `diff()`

In [ ]:
df_inz = df_kum.diff()

In [ ]:
df_inz.head()

In [ ]:
df_inz.tail()

## Fragestellung entwickeln

Viele verschiedene Auswertungen sind denkbar:
- verschiedene Zeitperioden vergleichen
- Inzidenz / totale Fallzahl
- verschiedene Kantone vergleichen
- verschiedene Grossregionen vergleichen
- Streuung unter den Kantonen / Grossregionen
- Fallzahlen pro Kopf
- Fallzahlen im Wochen-/Monatsrhythmus
- Gleitender Durchschnitt
- ...

## Auswertungen...